Minimizar $\boldsymbol{C}^T \boldsymbol x$ sujeto a
$$ Ax = b $$
$$ x \geq 0$$

In [215]:
import numpy as np
np.seterr(divide="ignore")

def switch_matrix_columns(A: np.ndarray, i1, i2) -> np.ndarray:
    A_new = A.copy()
    A_new[:, [i1, i2]] = A_new[:, [i2, i1]]

    return A_new

def simplex_iteracion(c: np.ndarray, A: np.ndarray, b: np.ndarray) -> dict:
    m, n = A.shape
    
    # Separar matriz A en variables básicas y no básicas
    A_b = A.copy()[:,0:m]
    A_n = A.copy()[:, m:n]
    
    #Costos c básicos y no básicos
    c_b = c.copy()[0, 0:m]
    c_n = c.copy()[0, m:n]
    
    try:
        A_b_inv = np.linalg.inv(A_b)
    except Exception as _:
        print('singular matrix')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_matrix_columns(A, i1, i2)
        return simplex_iteracion(c, A_new, b)
    
    # solución inicial z0 = z(x0)
    x0 = A_b_inv@b
    if any(x < 0 for x in x0):
        print('non basic solution')
        i1 = np.random.randint(0, n)
        i2 = np.random.randint(0, n)
        A_new = switch_matrix_columns(A, i1, i2)
        return simplex_iteracion(c, A_new, b)

    z0 = (c_b.T)@(A_b_inv@b)

    #variables duales y costos reducidos
    pi = c_b.T@A_b_inv
    c_red = pi@A_n - c_n.T

    return {
        "x": x0,
        "z": z0,
        "costos_reducidos": c_red,
        "Y": A_b_inv@A_n,
        "B_inv": A_b_inv
    }

In [224]:
def metodo_simplex_revisado(c: np.ndarray, A: np.ndarray, b:np.ndarray):
    m, _ = A.shape
    iter_inicial = simplex_iteracion(c, A, b)
    c_reducidos = iter_inicial["costos_reducidos"]
    Y = iter_inicial["Y"]
    B_inv = iter_inicial["B_inv"]

    if all(x <= 0 for x in c_reducidos):
        return iter_inicial
    
    idx_no_basica = np.argmax(c_reducidos)
    cocientes = np.divide(B_inv@b, np.vstack(Y[:, idx_no_basica]))
    idx_basica = np.argmin(np.abs(cocientes))
    i1, i2 = idx_basica, idx_no_basica + m
    
    A_new = A.copy()
    A_new[:, [i1, i2]] = A_new[:, [i2, i1]]
   
    c_new = c.copy()
    c_new[0, i1], c_new[0, i2] = c[0, i2], c[0, i1]

    return metodo_simplex_revisado(c_new, A_new, b)

    


# A = np.array([[1,0,0,1,0],[0,1,0,0,1],[0,0,1,3,2]])
# c = np.array([[0,0,0,-3,-5]])
# b = np.vstack([4, 6, 18])
A = np.array([[3, 2, 1, 0, 0], [1, 0, 0, 1, 0], [0, 1, 0, 0, 1]])
c = np.array([[-3,-5,0,0,0]])
b = np.vstack([18, 4, 6])
result = metodo_simplex_revisado(c,A,b)

for key, val in result.items():
    print(f'{key}: {val}')

non basic solution
non basic solution
singular matrix
x: [[18.]
 [ 4.]
 [ 6.]]
z: [-74.]
costos_reducidos: [ -6. -14.]
Y: [[2. 3.]
 [0. 1.]
 [1. 0.]]
B_inv: [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [55]:
c = [-3, -5]
A_ub = np.array([[3, 2]])
bounds = [(0, 4), (4,6)]

rows1=A_ub.shape[0]
rows2 = len(bounds)
rows3 = len(bounds)

rows_total = rows1+rows2+rows3

res = np.zeros(shape=(rows_total,))

In [34]:
a = np.identity(3)
b = np.vstack([1,2,3])

c = np.array([[0,0,0,-3,-5]])
c.copy()[0, 0:3]

array([0, 0, 0])